In [16]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder



In [17]:
df=pd.read_csv('final_data_for_modelling.csv')
df.head()
enc = OrdinalEncoder()
ohe    = OneHotEncoder(sparse=False)

enc.fit(df[["gender"]])
df[["gender"]] = enc.transform(df[["gender"]])
encoded_columns =    ohe.fit_transform(df[['appointment_day_of_week']])
data_hot_encoded = pd.DataFrame(encoded_columns, index=df.index)
data_hot_encoded.columns = ohe.get_feature_names(['appointment_day_of_week'])

data_other_cols = df.drop(columns='appointment_day_of_week')

#Concatenate the two dataframes :
df = pd.concat([data_hot_encoded, data_other_cols], axis=1)

df.head()



[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]]


/opt/homebrew/Caskroom/miniconda/base/envs/aipi510project/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,appointment_day_of_week_Friday,appointment_day_of_week_Monday,appointment_day_of_week_Saturday,appointment_day_of_week_Thursday,appointment_day_of_week_Tuesday,appointment_day_of_week_Wednesday,gender,age,neighbourhood,scholarship,...,age_group,tempmax,temp,feelslikemax,feelslike,humidity,windspeed,solarradiation,solarenergy,uvindex
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,62,JARDIM DA PENHA,0,...,seniors,75.2,73.3,75.2,73.3,76.7,16.1,52.9,4.3,2.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,56,JARDIM DA PENHA,0,...,adults,75.2,73.3,75.2,73.3,76.7,16.1,52.9,4.3,2.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,62,MATA DA PRAIA,0,...,seniors,75.2,73.3,75.2,73.3,76.7,16.1,52.9,4.3,2.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,8,PONTAL DE CAMBURI,0,...,children,75.2,73.3,75.2,73.3,76.7,16.1,52.9,4.3,2.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,56,JARDIM DA PENHA,0,...,adults,75.2,73.3,75.2,73.3,76.7,16.1,52.9,4.3,2.0


In [18]:
y=df['showed']
X=df.drop('showed', axis=1)
X=X[['gender','age','scholarship','hypertension','diabetes','alcoholism','handicap','sms_received','days_between_appointment_and_scheduled_day','tempmax','temp','feelslikemax','feelslike','humidity','windspeed','solarradiation','solarenergy','uvindex','appointment_day_of_week_Friday','appointment_day_of_week_Monday','appointment_day_of_week_Tuesday','appointment_day_of_week_Wednesday','appointment_day_of_week_Thursday','appointment_day_of_week_Friday','appointment_day_of_week_Saturday']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [19]:
model=RandomForestClassifier()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)

print(f'accuracy for test scores: {accuracy_score(y_test,y_pred):3f}')


accuracy for test scores: 0.778370
